In [3]:
!pip install chromadb sentence-transformers

  Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [15]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import uuid
from datetime import datetime


In [38]:
embedding_function = SentenceTransformerEmbeddingFunction(model_name = "all-MiniLM-L6-V2")
client = chromadb.Client()

COLLECTION_NAME = "agent_memory_final"

if COLLECTION_NAME in [c.name for c in client.list_collections()]:
  client.delete_collection(COLLECTION_NAME)

collection = client.get_or_create_collection(
    name= COLLECTION_NAME,
    embedding_function = embedding_function
)

In [46]:
def timestamp():
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [47]:
def store(role, message):
  collection.add(
      documents = [message],
      ids = [str(uuid.uuid4)],
      metadatas = [{"role": role, "timestamp":timestamp()}]
  )


In [48]:
def query_last_from(role, user_input):
  results = collection.query(query_texts= [user_input], n_results = 1, where = {"role":role})
  return results["documents"][0][0] if results ["documents"][0] else None

In [57]:
def show_history():
  results = collection.get(include= ["documents", "metadatas"])

  history = list(zip(results["metadatas"], results["documents"]))
  history.sort(key = lambda x: x[0]["timestamp"])
  print ("\n -- Chat History -- ")
  for meta,text in history:
    print(f"{meta['timestamp']} - {meta['role'].capitalize()}: {text}")

  print ("-----------------------\n")

In [58]:
def clear_history():
  client.delete_collection(COLLECTION_NAME)

  globals()["collection"]= client.get_or_create_collection(
    name= COLLECTION_NAME,
    embedding_function = embedding_function
  )

  print("Agent memory clear")

Agent is active. Type Messages.
Command: 'Show History', 'Query User', 'Clear History', 'exit' 



In [59]:
print("Agent is active. Type Messages.")
print ("Command: 'Show History', 'Query User', 'Clear History', 'exit' \n")
while True:
  user_input = input("You: ").strip()

  if user_input.lower() == "exit":
    print("Agent: Shutting down")
    break
  elif user_input.lower() == "show history":
    show_history()
    continue
  elif user_input.lower() == "query user":
    context  = query_last_from("user", "recall_something_user_said")
    print ("Agent: ", context if context else "No user-related memory found")
    continue
   elif user_input.lower() == "query agent":
    context  = query_last_from("user", "recall_something_user_said")
    print ("Agent: ", context if context else "No user-related memory found")
    continue
  elif user_input.lower() == "clear history":
    clear_history()
    continue

  store("user", user_input)

  recall = query_last_from("user", user_input)

  if recall and recall != user_input:
    reply = f"Previously you said something like: {recall}"
  else:
    reply = f"Please add more text"

  print ("Agent: ", reply)
  store ("agent", reply)



Agent is active. Type Messages.
Command: 'Show History', 'Query User', 'Clear History', 'exit' 

You: Show History

 -- Chat History -- 
2025-07-30 17:56:36 - User: sdsd
-----------------------

You: Clear History
Agent memory clear
You: How are you?


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Agent:  Please add more text
You: Show History

 -- Chat History -- 
2025-07-30 17:59:45 - User: How are you?
-----------------------

You: Query User
Agent:  How are you?
You: query user
Agent:  How are you?
You: clear history
Agent memory clear
You: query user
Agent:  No user-related memory found
You: hi 1
Agent:  Please add more text
You: hello dear
Agent:  Previously you said something like: hi 1
You: query user
Agent:  hi 1
You: show history

 -- Chat History -- 
2025-07-30 18:06:37 - User: hi 1
-----------------------



KeyboardInterrupt: Interrupted by user